# Unsupervised Topic Modeling

## Latent Dirichlet Allocation

Now the data we've been provided with is all unlabeled. To manually annotate these reviews would take awfully long and in general, is not practical. Hence, we need to look at unsupervised methods to label our data or atleast identify the different abstract topics we're dealing with. We perform LDA on our data to divide it into relevant topics. 

In [1]:
# Reading reviews 
import pandas as pd

data = pd.read_csv('./data.csv')

docs = data['text']

### Preprocessing
Now, we need to carry out some basic preprocessing to make our data suitable for the LDA model. The following few code cells will address just that.

In [2]:
# Using gensim to preprocess and tokenize the data 

import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = docs.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0])

C:\Users\praat\Unsupervised-Topic-Modeling\env\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


['tires', 'where', 'delivered', 'to', 'the', 'garage', 'of', 'my', 'choice', 'the', 'garage', 'notified', 'me', 'when', 'they', 'had', 'been', 'delivered', 'day', 'and', 'time', 'was', 'arranged', 'with', 'the', 'garage', 'and', 'went', 'and', 'had', 'them', 'fitted', 'hassel', 'free', 'experience']


In [3]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [4]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['redacted']) # Adding redacted as a stopword as it is not particularly relevant

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\praat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
!python -m spacy download en_core_web_sm

You should consider upgrading via the 'C:\Users\praat\Unsupervised-Topic-Modeling\env\Scripts\python.exe -m pip install --upgrade pip' command.



[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0])

['tire', 'deliver', 'garage', 'choice', 'garage', 'notify', 'deliver', 'day', 'time', 'arrange', 'garage', 'go', 'fit', 'hassel', 'free', 'experience']


In [8]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


This completes our preprocessing of the data. We can now feed this data into our LDA model. Now, LDA requires us to provide the number of topics we want the data to be segregated into. Initially, we try using an arbitrary value and see how a base model does. Later, we use a function to identify the number of topics that would be ideal. Here, we shall directly look at the custom function we made to identify the best parameters for our LDA model.  

In [ ]:
import tqdm
import numpy as np

# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

### Hyperparameter Tuning
There are three hyperparameters that we will try and tune to improve our LDA model. They are-
1) Number of topics \
2) Dirichlet hyperparameter alpha: Document-Topic Density \
3) Dirichlet hyperparameter beta: Word-Topic Density 

We run the code cell below and calculate the coherence for all combinations of hyperparameters. The higher the coherence, the better our model. 

In [ ]:
# Trying various 

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('/content/drive/MyDrive/lda_tuning_results.csv', index=False)
    pbar.close()

### Ideal Hyperparameters
After trying out a variety of hyperparameters, we see that the ideal number of topics is 5, alpha = 0.31 and eta is symmetric. For these values, we achieve a coherence of ~0.60 \
However, on trying a model with 5 topics, I felt that it did not cover a large number of words and the topics felt vague. Hence, I decided to try various number of topics and I found number of topics = 10 with alpha = 'asymmetric' and eta = 0.61 to provide more distinct topics with more interpretable and easily differentiable keywords. The model still manages to achieve a coherence of ~0.56 and hence, we will proceed with these hyperparameters for our model.


In [9]:
num_topics = 10

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=50,
                                           passes=5,
                                           alpha='asymmetric',
                                           eta=0.61)

In [10]:
from pprint import pprint

# Print the keywords in each topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.077*"service" + 0.059*"good" + 0.058*"price" + 0.047*"easy" + 0.044*"use" '
  '+ 0.042*"great" + 0.042*"tyre" + 0.029*"excellent" + 0.020*"garage" + '
  '0.019*"fit"'),
 (1,
  '0.058*"tyre" + 0.038*"time" + 0.031*"fit" + 0.029*"garage" + 0.021*"get" + '
  '0.017*"go" + 0.014*"car" + 0.013*"use" + 0.013*"day" + 0.012*"fitting"'),
 (2,
  '0.043*"start" + 0.042*"finish" + 0.022*"exactly" + 0.011*"smooth" + '
  '0.008*"perfect" + 0.008*"faultless" + 0.007*"process" + 0.005*"tin" + '
  '0.005*"say" + 0.005*"whole"'),
 (3,
  '0.043*"staff" + 0.037*"helpful" + 0.023*"friendly" + 0.018*"hassle_free" + '
  '0.011*"first_class" + 0.010*"polite" + 0.008*"guy" + 0.007*"knowledgeable" '
  '+ 0.004*"nice" + 0.004*"lovely"'),
 (4,
  '0.021*"anywhere_else" + 0.012*"wide" + 0.008*"seamless" + 0.008*"class" + '
  '0.007*"sell" + 0.006*"top" + 0.005*"grip" + 0.005*"try" + 0.005*"past" + '
  '0.004*"noise"'),
 (5,
  '0.045*"tyre" + 0.017*"car" + 0.012*"order" + 0.011*"fit" + 0.009*"size" + '
  '

## Visualizing topic clusters
We can visualize the generated topic clusters using pyLDAvis and see the various keywords for each topic.

In [12]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

C:\Users\praat\Unsupervised-Topic-Modeling\env\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.345481 -0.151435       1        1  44.772135
1      0.236011  0.103521       2        1  29.947291
7      0.052401  0.160850       3        1   9.248015
5      0.061236  0.036675       4        1   7.255088
3     -0.114348 -0.055102       5        1   1.972895
2     -0.116937 -0.037019       6        1   1.758231
4     -0.107867 -0.019155       7        1   1.680179
9     -0.123468 -0.008821       8        1   1.654928
6     -0.114643 -0.016220       9        1   1.135696
8     -0.117866 -0.013293      10        1   0.575544, topic_info=           Term         Freq        Total Category  logprob  loglift
14         easy  2409.000000  2409.000000  Default  30.0000  30.0000
28        price  3057.000000  3057.000000  Default  29.0000  29.0000
31         tyre  4567.000000  4567.000000  Default  28.0000  28.0000
20      service  4235.000000  4235.000000  Default  27.0000  27.0000
21         good  3310.000000  3310.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2642   magazine     0.461957     6.019854  Topic10  -7.2555   2.5903
1819     winter     0.697665    19.628344  Topic10  -6.8432   1.8206
1426      crack     0.536023     9.319161  Topic10  -7.1068   2.3020
1385     amount     0.582612    16.552787  Topic10  -7.0234   1.8108
2577  wonderful     0.464308    12.019762  Topic10  -7.2504   1.9039

[516 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1333      1  0.041551       accept
1333      2  0.083103       accept
1333      3  0.789474       accept
1333      4  0.041551       accept
2322      1  0.038957  accommodate
...     ...       ...          ...
363       4  0.630951        wrong
4263      1  0.243829     youfitte
4263      2  0.243829     youfitte
4263      3  0.243829     youfitte
4263     10  0.243829     youfitte

[1955 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 8, 6, 4, 3, 5, 10, 7, 9])

Now, let's save our model and further explore how to proceed with labeling in the next notebook.

In [11]:
lda_model.save('lda.model')

In [12]:
# Storing these variables as they will be required later on while making predictions in our next notebook
%store stop_words
%store bigram_mod
%store id2word
%store nlp

Stored 'stop_words' (list)
Stored 'bigram_mod' (FrozenPhrases)
Stored 'id2word' (Dictionary)
Stored 'nlp' (English)
